In [ ]:
import os
os.getcwd()
# os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\sleigh_the_model


In [29]:
import pandas as pd
import scripts.features as ft
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso


In [4]:
file = 'data/dataformodel.csv'
months_in_adv = 12
months_lag = 12
data = ft.load_data(file, months_in_adv)


In [5]:
features_to_lag = [
    'pce',
    'inflation',
    'unemployment_rate',
    'revolving_credit',
    'nonrevolving_credit',
    'total_credit'
]

In [ ]:
data_with_lag = ft.add_lag_features(data, months_lag, features_to_lag)

In [17]:
subset_cols = [col for col in data_with_lag.columns if ('lag' in col) or (col in features_to_lag) or (col == 'pce_future')]
train_data = data_with_lag.dropna(subset=subset_cols)

X_train_columns = [col for col in train_data.columns if col not in ['date', 'pce_future', 'pce']]
X_train = train_data[X_train_columns]
y_train = train_data['pce_future']

In [30]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

lasso =  Lasso(alpha=0.1, max_iter=10000)
lasso.fit(X_scaled, y_train)

c:\Users\blond\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.330e+07, tolerance: 2.246e+06
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1, max_iter=10000)

In [31]:
last_date = data_with_lag['date'].max()

prediction_dates = pd.date_range(
    start = last_date + pd.DateOffset(months = 1),
    end = '2026-01-01',
    freq = 'MS'
).strftime('%Y-%m-%d').tolist()
print(last_date)
print(prediction_dates)

2025-04-01 00:00:00
['2025-05-01', '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01', '2025-11-01', '2025-12-01', '2026-01-01']


In [41]:
future_features_df = pd.DataFrame()
prediction_output_dates = []

last_known_date = data_with_lag.sort_values('date').iloc[-1]
current_row = last_known_date
pred_rows = []

future_dates = pd.date_range(
    start = last_date + pd.DateOffset(months = 1),
    end ='2026-01-01',
    freq = 'MS'
)

In [ ]:
for forecast_date in future_dates:
    # adding data for features from the prior month for data not available in the future
    for feature in features_to_lag:
        for i in range(months_in_adv, 1, -1):
            current_row[f"{feature}_lag_{i}"] = current_row[f"{feature}_lag_{i-1}"]
        current_row[f"{feature}_lag_1"] = current_row[feature]
    current_row['date'] = forecast_date

    predict_input = pd.DataFrame([current_row[X_train_columns]])
    predict_input_scaled = scaler.transform(predict_input)
    pred = lasso.predict(predict_input_scaled)[0]

    pred_rows.append({
        'Prediction_Date': forecast_date,
        'Predicted_PCE': pred       
    })

    current_row['pce'] = pred

prediction_results = pd.DataFrame(pred_rows)
# adding smoothed result since the prediction seemed a little high and increased every month.
prediction_results['Smoothed_PCE'] = prediction_results['Predicted_PCE'].rolling(window = 3, min_periods = 1).mean()
print("*** Future PCE Predictions ***")
print(prediction_results)

*** Future PCE Predictions ***
   Prediction_Date  Predicted_PCE   Smoothed_PCE
0       2025-05-01   22166.509109   22166.509109
1       2025-06-01   23685.745425   22926.127267
2       2025-07-01   25578.912051   23810.388862
3       2025-08-01   28130.968682   25798.542053
4       2025-09-01   31493.029088   28400.969940
5       2025-10-01   35968.197988   31864.065253
6       2025-11-01   41969.393263   36476.873446
7       2025-12-01   50059.644844   42665.745365
8       2026-01-01   60948.844569   50992.627559
9       2025-05-01   75513.568335   62174.019249
10      2025-06-01   94980.817007   77147.743304
11      2025-07-01  120743.890788   97079.425377
12      2025-08-01  154814.228276  123512.978690
13      2025-09-01  200018.642722  158525.587262
14      2025-10-01  259964.698000  204932.522999
15      2025-11-01  339441.489657  266474.943460
16      2025-12-01  444809.508832  348071.898830
17      2026-01-01  584471.867321  456240.955270
